<a href="https://colab.research.google.com/github/Aditya-hacks/Crime-record-management/blob/main/fraud_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers datasets torch scikit-learn pandas



In [23]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [24]:
dataset = load_dataset("sms_spam")

dataset


DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [25]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [26]:
def tokenize_function(example):
    return tokenizer(
        example["sms"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [27]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [28]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]


KeyError: 'test'

In [33]:
# Create train/test split manually (80/20)
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

dataset

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 3567
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 892
    })
})

In [34]:
def tokenize_function(example):
    return tokenizer(
        example["sms"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3567 [00:00<?, ? examples/s]

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

In [41]:
tokenized_dataset.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]


In [42]:
print(tokenized_dataset["train"].column_names)

['sms', 'labels', 'input_ids', 'attention_mask']


In [45]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.to(device)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


NameError: name 'device' is not defined

In [46]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [47]:
import torch
from transformers import DistilBertForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.to(device)


Using device: cpu


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSelfAttention(
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [50]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [52]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True
)


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [54]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [55]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.049285,0.982063,0.905512,0.966387,0.934959
2,No log,0.032708,0.994395,0.991379,0.966387,0.978723
3,0.048099,0.040205,0.988789,0.950413,0.966387,0.958333


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=669, training_loss=0.039187297336367395, metrics={'train_runtime': 7247.9278, 'train_samples_per_second': 1.476, 'train_steps_per_second': 0.092, 'total_flos': 354383408254464.0, 'train_loss': 0.039187297336367395, 'epoch': 3.0})

In [56]:
results = trainer.evaluate()
print(results)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.032700661569833755, 'eval_accuracy': 0.9943946188340808, 'eval_precision': 0.9913793103448276, 'eval_recall': 0.9663865546218487, 'eval_f1': 0.9787234042553191, 'eval_runtime': 211.8575, 'eval_samples_per_second': 4.21, 'eval_steps_per_second': 0.264, 'epoch': 3.0}


In [59]:
def predict_sms(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.argmax(outputs.logits).item()
    probability = torch.softmax(outputs.logits, dim=1)[0][prediction].item()

    if prediction == 1:
        print(f"🚨 Spam detected (confidence: {probability:.4f})")
    else:
        print(f"✅ Safe message (confidence: {probability:.4f})")

In [60]:
predict_sms("Hey, are we meeting at 6pm today?")

✅ Safe message (confidence: 0.9987)


In [61]:
predict_sms("Let's meet at 7pm near metro station.")

✅ Safe message (confidence: 0.9990)


In [62]:
predict_sms("Dear customer, your UPI account will be suspended. Verify immediately at secureupi-update.com")

🚨 Spam detected (confidence: 0.6796)


In [63]:
predict_sms("Congratulations! You have won $5000. Click here to claim now.")

✅ Safe message (confidence: 0.8573)


In [64]:
indian_scam_data = [
    # UPI / Bank Fraud
    ("Dear customer, your UPI account will be suspended. Verify immediately at secureupi-update.com", 1),
    ("Your SBI account is blocked. Update KYC now to avoid suspension.", 1),
    ("Your bank account has unusual activity. Click here to secure immediately.", 1),
    ("Refund of Rs 4999 processed. Approve request in UPI to receive money.", 1),
    ("You have won IPL lottery 25 lakh rupees. Claim now urgently.", 1),
    ("RBI notice: Your account will be frozen within 24 hours.", 1),
    ("Courier delivery failed. Pay Rs 25 to reschedule now.", 1),
    ("Your PAN card will be deactivated. Update details immediately.", 1),
    ("Activate your credit card reward points before expiry.", 1),
    ("Dear Paytm user, verify wallet KYC at paytm-secure-update.com", 1),

    # Normal Indian messages
    ("Hey, I'll transfer money via UPI tonight.", 0),
    ("Meeting shifted to 4pm tomorrow.", 0),
    ("Please send the invoice PDF.", 0),
    ("Dinner at 8pm?", 0),
    ("Recharge my number with 199 plan.", 0),
    ("UPI transfer done, check once.", 0),
    ("Happy birthday bhai!", 0),
    ("Train is delayed by 20 minutes.", 0)
]

In [65]:
import pandas as pd
from datasets import Dataset

df_indian = pd.DataFrame(indian_scam_data, columns=["sms", "labels"])

dataset_indian = Dataset.from_pandas(df_indian)

In [67]:
# Convert original raw dataset (before tokenization) to pandas
df_original = pd.DataFrame({
    "sms": dataset["train"]["sms"],
    "labels": dataset["train"]["label"]   # <-- use label, not labels
})

In [68]:
df_combined = pd.concat([df_original, df_indian], ignore_index=True)

from datasets import Dataset
dataset_combined = Dataset.from_pandas(df_combined)

In [69]:
dataset_combined = dataset_combined.train_test_split(test_size=0.2, seed=42)

In [70]:
tokenized_combined = dataset_combined.map(tokenize_function, batched=True)

Map:   0%|          | 0/2868 [00:00<?, ? examples/s]

Map:   0%|          | 0/717 [00:00<?, ? examples/s]

In [71]:
tokenized_combined.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

train_dataset_new = tokenized_combined["train"]
test_dataset_new = tokenized_combined["test"]

In [72]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.to(device)

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSelfAttention(
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [73]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_new,
    eval_dataset=test_dataset_new,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.038354,0.990237,0.987013,0.926829,0.955975
2,No log,0.040678,0.988842,0.940476,0.963415,0.951807
3,0.051194,0.040786,0.993026,0.975309,0.963415,0.969325


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=540, training_loss=0.04758721763888995, metrics={'train_runtime': 5865.913, 'train_samples_per_second': 1.467, 'train_steps_per_second': 0.092, 'total_flos': 284937374509056.0, 'train_loss': 0.04758721763888995, 'epoch': 3.0})

In [74]:
def predict_sms(text, threshold=0.6):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.softmax(outputs.logits, dim=1)[0]
    spam_prob = probs[1].item()

    if spam_prob > threshold:
        print(f"🚨 Spam detected (confidence: {spam_prob:.4f})")
    else:
        print(f"✅ Safe message (confidence: {1-spam_prob:.4f})")

In [75]:
predict_sms("Your bank account is blocked. Update immediately.")

🚨 Spam detected (confidence: 0.6181)


In [76]:
predict_sms("Dear customer, your UPI account will be suspended.")

✅ Safe message (confidence: 0.9938)


In [77]:
more_indian_scams = [
    ("Your UPI account has been temporarily suspended due to suspicious activity.", 1),
    ("Immediate action required. Your bank account will be frozen.", 1),
    ("Verify your UPI ID within 12 hours to avoid permanent block.", 1),
    ("RBI warning: Update your KYC now to continue services.", 1),
    ("Payment failed. Click link to resolve UPI issue.", 1),
    ("Your PhonePe account will expire today. Verify now.", 1),
    ("Google Pay alert: Unusual login detected.", 1),
    ("Axis Bank notice: Account suspended due to incomplete KYC.", 1),
    ("Refund of Rs 5000 pending. Approve request immediately.", 1),
    ("Your account will be deactivated if not verified today.", 1),
]

In [78]:
df_more = pd.DataFrame(more_indian_scams, columns=["sms", "labels"])
df_combined = pd.concat([df_combined, df_more], ignore_index=True)

In [79]:
dataset_combined = Dataset.from_pandas(df_combined)
dataset_combined = dataset_combined.train_test_split(test_size=0.2, seed=42)
tokenized_combined = dataset_combined.map(tokenize_function, batched=True)

tokenized_combined.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

train_dataset_new = tokenized_combined["train"]
test_dataset_new = tokenized_combined["test"]

Map:   0%|          | 0/2876 [00:00<?, ? examples/s]

Map:   0%|          | 0/719 [00:00<?, ? examples/s]

In [80]:
predict_sms("Dear customer, your UPI account will be suspended.", threshold=0.5)

✅ Safe message (confidence: 0.9938)


In [81]:
def hybrid_predict(text):
    suspicious_words = [
        "upi", "kyc", "suspended", "blocked",
        "verify", "rbi", "refund", "account",
        "urgent", "deactivated"
    ]

    rule_score = 0
    for word in suspicious_words:
        if word in text.lower():
            rule_score += 0.08

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    ml_prob = torch.softmax(outputs.logits, dim=1)[0][1].item()

    final_score = min(ml_prob + rule_score, 1.0)

    if final_score > 0.5:
        print(f"🚨 Spam detected (combined score: {final_score:.4f})")
    else:
        print(f"✅ Safe message (combined score: {final_score:.4f})")

In [82]:
hybrid_predict("Dear customer, your UPI account will be suspended.")

✅ Safe message (combined score: 0.2462)


In [83]:
hybrid_predict("Your bank account is blocked. Update immediately.")

🚨 Spam detected (combined score: 0.7781)


In [84]:
hybrid_predict("Hey, are we meeting at 6pm?")

✅ Safe message (combined score: 0.0025)


In [85]:
hybrid_predict("UPI verification required within 24 hours.")

✅ Safe message (combined score: 0.0968)


In [86]:
hybrid_predict("Refund of Rs 5000 pending in your account.")

🚨 Spam detected (combined score: 1.0000)


In [88]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.03830355778336525,
 'eval_accuracy': 0.9902370990237099,
 'eval_precision': 0.987012987012987,
 'eval_recall': 0.926829268292683,
 'eval_f1': 0.9559748427672956,
 'eval_runtime': 139.1225,
 'eval_samples_per_second': 5.154,
 'eval_steps_per_second': 0.323,
 'epoch': 3.0}

In [89]:
hybrid_predict("Dear customer, your UPI account will be suspended.")

✅ Safe message (combined score: 0.2462)
